In [1]:
#-*- coding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

from sklearn.externals import joblib
from konlpy.tag import Mecab
from konlpy.utils import pprint

def data_add(text):
    text = text.replace(" ", ',,,,')
    return text

In [2]:
clf = joblib.load('classify.model')
cate_dict = joblib.load('cate_dict.dat')
vectorizer = joblib.load('vectorizer.dat')

In [3]:
joblib.dump(clf,'n_classify.model')

['n_classify.model',
 'n_classify.model_01.npy',
 'n_classify.model_02.npy',
 'n_classify.model_03.npy']

In [4]:
joblib.dump(cate_dict,'n_cate_dict.dat')
joblib.dump(vectorizer,'n_vectorizer.dat')

['n_vectorizer.dat', 'n_vectorizer.dat_01.npy', 'n_vectorizer.dat_02.npy']

In [5]:
mecab = Mecab()

In [6]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

# ---

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import pandas as pd

In [8]:
train_df = pd.read_pickle("soma_goods_train.df")

In [9]:
d_list = []
cate_list = []
for each in train_df.iterrows():
    cate = ";".join([each[1]['cate1'],each[1]['cate2'],each[1]['cate3']])
    d_list.append(" ".join(str(e) for e in mecab.pos(data_add(each[1]['name']))))
    cate_list.append(cate)

In [10]:
print len(set(cate_list))

In [12]:
cate_dict = dict(zip(list(set(cate_list)),range(len(set(cate_list)))))

In [11]:
y_list = []
for each in train_df.iterrows():
    cate = ";".join([each[1]['cate1'],each[1]['cate2'],each[1]['cate3']])
    y_list.append(cate_dict[cate])

In [13]:
x_list = vectorizer.fit_transform(d_list)

In [14]:
x_list.shape

(10000, 254527)

In [15]:
test = SelectKBest(chi2,k=120000)
fit = test.fit(x_list,y_list)

In [16]:
data =" ".join(str(e) for e in mecab.pos(data_add('[신한카드5%할인][서우한복] 아동한복 여자아동 금나래 (분홍)')))
data = fit.transform(vectorizer.transform([data]))

In [17]:
##data =" ".join(str(e) for e in mecab.pos(data_clean_adnormal2('[신한카드5%할인][서우한복] 아동한복 여자아동 금나래 (분홍)')))
#pred = clf.predict(vectorizer.transform([data]))[0]
pred = clf.predict(data)[0]
#data.shape

In [ ]:
# normal
from bottle import route, run, template,request,get, post


import  time
from threading import  Condition
_CONDITION = Condition()
@route('/classify')
def classify():
    pprint (u"classify")
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    print name
    pred = clf.predict(vectorizer.transform([" ".join(str(e) for e in mecab.pos(data_clean_adnormal2(name)))]))[0]
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)


In [ ]:
# selectKBest 
from bottle import route, run, template,request,get, post



import  time
from threading import  Condition
_CONDITION = Condition()
@route('/classify')
def classify():
    pprint (u"classify")
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    print name
    pred = clf.predict(fit.transform(
            vectorizer.transform(
                [" ".join(str(e) for e in mecab.pos(data_add(name)))])
        ))[0]
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)


 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능